In [ ]:
import syft_runtimes.high_low as syhl
import syft_datasets as syd
from syft_notebook_ui.utils import show_dir

## High and Low Configs

DO's email

In [ ]:
email = "flower-test-group-2@openmined.org"

The connection config (SSH in this case) to connect to the low datasite

In [ ]:
from pathlib import Path

private_key_path = Path("~/.ssh/flower-vm-2").expanduser()
assert private_key_path.exists()

ssh_config: dict = {
    "host": "48.216.217.94",
    "user": "azureuser",
    "port": 22,
    "ssh_key_path": private_key_path,
}

The name of the high low runtime

In [ ]:
highlow_identifier = "highlow-1234"

## Initialize the high datasite (secure, self-contained)

In [ ]:
highside_client: syhl.HighSideClient = syhl.initialize_high_datasite(
    email=email,
    highlow_identifier=highlow_identifier,
    lowside_syftbox_dir="/home/azureuser/SyftBox",  # the location of the SyftBox dir on the low side
    connection_config=ssh_config,
    force_overwrite=True,
)

In [ ]:
highside_client.config_path

In [ ]:
show_dir(highside_client.workspace.data_dir.parents[1])

## Create dataset on high side

In [ ]:
DATASET_NAME = "my_cool_dataset"
data_dir = Path("./data")

highside_dataset = syd.create(
    name=DATASET_NAME,
    mock_path=data_dir / "mock_data.csv",
    private_path=data_dir / "private_data.csv",
    readme_path=data_dir / "readme.md",
    syftbox_config_path=highside_client.config_path,
)

In [ ]:
highside_dataset.private_dir

In [ ]:
show_dir(highside_client.workspace.data_dir.parents[1])

## Sync dataset to low side

The connection to the low side was automatically configured during initialization using the `ssh_config` dictionary.

In [ ]:
# Sync the dataset (mock part) to the low side
highside_client.sync_dataset(
    dataset_name=DATASET_NAME,
    verbose=True,
)

## Sync another dataset to the low side

In [ ]:
DATASET_NAME = "my_cool_dataset_2"
data_dir = Path("./data")

highside_dataset = syd.create(
    name=DATASET_NAME,
    mock_path=data_dir / "mock_data.csv",
    private_path=data_dir / "private_data.csv",
    readme_path=data_dir / "readme.md",
    syftbox_config_path=highside_client.config_path,
)

# Sync the dataset (mock part) to the low side
highside_client.sync_dataset(
    dataset_name=DATASET_NAME,
    verbose=True,
)

## Sync the pending jobs from the low side

Go to the low side VM and accept some jobs to the `SyftBox/private/<email>/syft_runtimes/highlow-1234/jobs`

In [ ]:
highside_client.sync_pending_jobs()

In [ ]:
show_dir(highside_client.runtime_dir)

## Sync the done jobs to the low side

Assume the high side client runs the `job01` and output the results to `done/job01`

In [ ]:
(highside_client.runtime_dir / "done" / "job01").mkdir(exist_ok=True)
(highside_client.runtime_dir / "done" / "job01" / "result.json").touch()

In [ ]:
show_dir(highside_client.runtime_dir)

In [ ]:
highside_client.sync_done_jobs()